# Create an interface with no strain matching

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set slab parameters for the substrate and film in cell 2.1. (or use default).
1. Set interface parameters in cell 3.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create substrate and film slabs and select the terminations
1. Generate an interface by wrapping the film slab into the substrate slab. Optionally, the film slab can be scaled to match the substrate lattice.
1. Select the interface with the desired strain and visualize it

## Notes
1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare Environment
### 1.1. Set notebook parameters

In [65]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 
# Enable scaling of the film slab atomic coordinates to match the substrate lattice (preserve coordinates in crystal units).
ENABLE_FILM_SCALING = True

# Create slabs for the film and substrate when creating the interface, otherwise use provided materials directly.
CREATE_SLABS = True 

FILM_INDEX = 1
FILM_MILLER_INDICES = (0, 0, 1)
FILM_THICKNESS = 1  # in atomic layers
FILM_VACUUM = 0.0  # in angstroms
FILM_TERMINATION_FORMULA = None  # if None, the first termination will be used
FILM_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
FILM_USE_ORTHOGONAL_C = True
FILM_XY_SHIFT = [0.0, 0.0]  # in Angstroms, shift the film slab in the XY plane relative to the substrate slab

SUBSTRATE_INDEX = 0
SUBSTRATE_MILLER_INDICES = (1, 1, 1)
SUBSTRATE_THICKNESS = 3  # in atomic layers
SUBSTRATE_VACUUM = 3.0  # in angstroms
SUBSTRATE_TERMINATION_FORMULA = None  # if None, the first termination will be used
SUBSTRATE_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
SUBSTRATE_USE_ORTHOGONAL_C = True

# Whether to convert materials to conventional cells before creating slabs.
USE_CONVENTIONAL_CELL = False


# Set the termination pair indices
TERMINATION_PAIR_INDEX = 0 # Will be overridden in interactive selection is used
INTERFACE_DISTANCE = 3.0  # in Angstrom
INTERFACE_VACUUM = 20.0  # in Angstrom


### 1.2. Install Packages
Note: the step executes only when using Pyodide. For other environments (e.g. Colab, JupyterLab), the packages should be installed via `pip install .` (see [README](../../README.ipynb)).

In [66]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")

### 1.3. Get input materials and assign `substrate` and `film`
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [67]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())
substrate = materials[SUBSTRATE_INDEX]
try: 
    film = materials[FILM_INDEX]
except IndexError:
    print("Please select a film material. Film is set to substrate.")
    film = substrate

0: 0-Ni
1: 1-Graphene
2: C(001)-Ni(001), Interface, Strain 0.402pct
3: C(001)-Ni(001), Interface, Strain 30.393pct
4: C(001)-Ni(011), Interface, Strain 30.947pct
5: C(001)-Ni(100), Interface, Strain 0.402pct
6: C(001)-Ni(100), Interface, Strain 30.393pct
7: C(001)-Ni(111), Interface, Strain 0.402pct
8: C(001)-Ni(111), Interface, Strain 71.515pct
9: C, Graphene, HEX (P6:mmm) 2D (Monolayer), 2dm-3993 - Armchair Nanoribbon (11)
10: C2(001)-Ni4(001), Interface
11: Custom Shape - Etching Pattern
12: Ga2 Te2
13: Ni(001), termination Ni_P4:mmm_2, Slab
14: Ni(001)-Ni(111), Grain Boundary
15: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 (Perturbation: 0.05*sin(2*pi*x)*sin(2*pi*y))
16: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 (Perturbation: 0.5*sin(2*pi*x))
17: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 (Perturbation: 0.5*sin(2*pi*y))
18: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 Cluster R=4.5A
19: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23 Octahedron cluster
20: Ni, Nickel, FCC (Fm-3m) 3D (Bulk), mp-23,

### 1.4. Preview Substrate and Film

In [68]:
from utils.visualize import visualize_materials as visualize
visualize([substrate, film], repetitions=[3, 3, 1], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Ni - Material - rotation: 0x', layout=Layout(align_self='center'…

## 2. Create slabs for the interface



### 2.2. Get possible terminations for the slabs

In [69]:
from mat3ra.made.tools.build.slab.helpers import get_slab_terminations

film_slab_terminations = get_slab_terminations(film, miller_indices=FILM_MILLER_INDICES)
substrate_slab_terminations = get_slab_terminations(substrate, miller_indices=SUBSTRATE_MILLER_INDICES)

### 2.3. Visualize slabs for all possible terminations

In [70]:
from mat3ra.made.tools.analyze.lattice_planes import CrystalLatticePlanesMaterialAnalyzer
from mat3ra.made.tools.build.slab.termination_utils import select_slab_termination

film_analyzer = CrystalLatticePlanesMaterialAnalyzer(material=film, miller_indices=FILM_MILLER_INDICES)
film_slabs = [film_analyzer.get_material_with_termination_without_vacuum(termination) for termination in
              film_slab_terminations]

substrate_analyzer = CrystalLatticePlanesMaterialAnalyzer(material=substrate, miller_indices=SUBSTRATE_MILLER_INDICES)
substrate_slabs = [substrate_analyzer.get_material_with_termination_without_vacuum(termination) for termination in
                   substrate_slab_terminations]

film_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                          zip(film_slabs, film_slab_terminations)]
substrate_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in
                               zip(substrate_slabs, substrate_slab_terminations)]

visualize(film_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")
visualize(substrate_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='C2 - C_P6/mmm_2 - rotation: -90x', layout=Layout(align_self='cen…

GridBox(children=(VBox(children=(Label(value='Ni4 - Ni_R-3m_4 - rotation: -90x', layout=Layout(align_self='cen…

### 2.3. Select terminations for the Slabs

In [71]:
film_termination = select_slab_termination(film_slab_terminations, FILM_TERMINATION_FORMULA)
substrate_termination = select_slab_termination(substrate_slab_terminations, SUBSTRATE_TERMINATION_FORMULA)

### 2.1. Create Substrate and Layer Slabs
Slab Configuration lets define the slab thickness, vacuum, and the Miller indices of the interfacial plane and get the slabs with possible terminations.
Define the substrate slab cell that will be used as a base for the interface and the film slab cell that will be placed on top of the substrate slab.

In [72]:
from mat3ra.made.tools.build.slab.helpers import create_slab

if CREATE_SLABS:
    film_slab = create_slab(
        crystal=film,
        miller_indices=FILM_MILLER_INDICES,
        termination=film_termination,
        number_of_layers=FILM_THICKNESS, # in atomic layers
        vacuum=FILM_VACUUM, # in angstroms
        xy_supercell_matrix=FILM_XY_SUPERCELL_MATRIX,
        use_orthogonal_c=FILM_USE_ORTHOGONAL_C,
        use_conventional_cell=USE_CONVENTIONAL_CELL
    )

    substrate_slab = create_slab(
        crystal=substrate,
        miller_indices=SUBSTRATE_MILLER_INDICES,
        termination=substrate_termination,
        number_of_layers=SUBSTRATE_THICKNESS, # in atomic layers
        vacuum=SUBSTRATE_VACUUM, # in angstroms
        xy_supercell_matrix=SUBSTRATE_XY_SUPERCELL_MATRIX,
        use_orthogonal_c=SUBSTRATE_USE_ORTHOGONAL_C,
        use_conventional_cell=USE_CONVENTIONAL_CELL
    )
else:
    film_slab = film
    substrate_slab = substrate


## 3. Create an interface
### 3.1. Create an interface with no strain matching
We use no strain matching for the interface construction. Substrate slab cell is used as a base and the film slab is placed on top of it with the specified distance.

In [73]:
from mat3ra.made.tools.build.interface.helpers import  create_simple_interface_between_slabs

interface = create_simple_interface_between_slabs(
    film_slab=film_slab,
    substrate_slab=substrate_slab,
    gap =INTERFACE_DISTANCE,
    vacuum=INTERFACE_VACUUM,
    xy_shift=FILM_XY_SHIFT
)

In [74]:
visualize([interface], repetitions=[3, 3, 1])
visualize([interface], repetitions=[3, 3, 1], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='C8Ni12 - Material - rotation: 0x,0y,0z', layout=Layout(align_sel…

GridBox(children=(VBox(children=(Label(value='C8Ni12 - Material - rotation: -90x', layout=Layout(align_self='c…

## 5. Pass data to the outside runtime

In [75]:
from utils.jupyterlite import set_materials
set_materials(interface)

Data for materials written to uploads/C(001)-Ni(111), Interface, Strain 71.515pct.json
